In [24]:
import torch
import torch.nn as nn

# Set manual seed
torch.manual_seed(2)

from sklearn import datasets
from sklearn import preprocessing
iris = datasets.load_iris()
X = torch.tensor(preprocessing.normalize(iris.data[:, :2]), dtype=torch.float)
y = torch.tensor(iris.target.reshape(-1, 1), dtype=torch.float)

# We only take 2 classes to make a binary classification problem
X = X[:y[y < 2].size()[0]]
y = y[:y[y < 2].size()[0]]


In [25]:
class FNN(nn.Module):
    def __init__(self, ):
        super().__init__()

        # Dimensions for input, hidden and output
        self.input_dim = 2
        self.hidden_dim = 32
        self.output_dim = 1

        # Learning rate definition
        self.learning_rate = 0.001

        # Our parameters (weights)
        # w1: 2 x 32
        self.w1 = torch.randn(self.input_dim, self.hidden_dim)

        # w2: 32 x 1 ##outputdim--> hiddendim
        ##self.w2 = torch.randn(self.hidden_dim, self.output_dim)
        self.w2 = torch.randn(self.hidden_dim, self.hidden_dim)
        
        ##LAYER 하나 추가
        ## w3
        self.w3 = torch.randn(self.hidden_dim, self.output_dim)

    def sigmoid(self, s):
        return 1 / (1 + torch.exp(-s))

    def sigmoid_first_order_derivative(self, s):
        return s * (1 - s)

    # Forward propagation
    def forward(self, X):
        # First linear layer
        self.y1 = torch.matmul(X, self.w1) # 3 X 3 ".dot" does not broadcast in PyTorch

        # First non-linearity
        self.y2 = self.sigmoid(self.y1)

        # Second linear layer
        self.y3 = torch.matmul(self.y2, self.w2)

        # Second non-linearity
        ##y4를 self.y4로 
        self.y4 = self.sigmoid(self.y3)
        
        ## Third linear layer
        self.y5 = torch.matmul(self.y4, self.w3)
        
        ## Third non-linearity
        y6 = self.sigmoid(self.y5)
        
        ##y4 --> y6
        return y6
    
    ##y4-->y6
    # Backward propagation
    def backward(self, X, l, y6):
        # Derivative of binary cross entropy cost w.r.t. final output y4
        ##y4-->y6
        self.dC_dy6 = y6 - l
        
    

        ##
        '''
        Gradients for w3: partial derivative of cost w.r.t. w2
        dC/dw2
        '''
        ##y4-->y6
        self.dy6_dy5 = self.sigmoid_first_order_derivative(y6)
        self.dy5_dw3 = self.y4
        
        ##y4-->y6
        # Y4 delta: dC_dy4 dy4_dy3
        self.y6_delta = self.dC_dy6 * self.dy6_dy5
        
        ##y4-->y6
        # This is our gradients for w1: dC_dy4 dy4_dy3 dy3_dw2
        self.dC_dw3 = torch.matmul(torch.t(self.dy5_dw3), self.y6_delta)

        
        '''
        Gradients for w2: partial derivative of cost w.r.t. w2
        dC/dw2
        '''
        self.dy5_dy4 = self.w3
        self.dy4_dy3 = self.sigmoid_first_order_derivative(self.y4)
        self.dy3_dw2 = self.y2

        # Y2 delta: (dC_dy4 dy4_dy3) dy3_dy2 dy2_dy1
        self.y4_delta = torch.matmul(self.y6_delta, torch.t(self.dy5_dy4)) * self.dy4_dy3

        # Gradients for w1: (dC_dy4 dy4_dy3) dy3_dy2 dy2_dy1 dy1_dw1
        self.dC_dw2 = torch.matmul(torch.t(self.dy3_dw2), self.y4_delta)

        '''
        Gradients for w1: partial derivative of cost w.r.t w1
        dC/dw1
        '''
        self.dy3_dy2 = self.w2
        self.dy2_dy1 = self.sigmoid_first_order_derivative(self.y2)

        # Y2 delta: (dC_dy4 dy4_dy3) dy3_dy2 dy2_dy1
        self.y2_delta = torch.matmul(self.y4_delta, torch.t(self.dy3_dy2)) * self.dy2_dy1

        # Gradients for w1: (dC_dy4 dy4_dy3) dy3_dy2 dy2_dy1 dy1_dw1
        self.dC_dw1 = torch.matmul(torch.t(X), self.y2_delta)

        # Gradient descent on the weights from our 2 linear layers
        self.w1 -= self.learning_rate * self.dC_dw1
        self.w2 -= self.learning_rate * self.dC_dw2
        self.w3 -= self.learning_rate * self.dC_dw3

    def train(self, X, l):
        # Forward propagation
        y6 = self.forward(X)

        # Backward propagation and gradient descent
        self.backward(X, l, y6)

In [26]:
# Instantiate our model class and assign it to our model object
model = FNN()

# Loss list for plotting of loss behaviour
loss_lst = []

# Number of times we want our FNN to look at all 100 samples we have, 100 implies looking through 100x
num_epochs = 101

# Let's train our model with 100 epochs
for epoch in range(num_epochs):
    # Get our predictions
    y_hat = model(X)

    # Cross entropy loss, remember this can never be negative by nature of the equation
    # But it does not mean the loss can't be negative for other loss functions
    cross_entropy_loss = -(y * torch.log(y_hat) + (1 - y) * torch.log(1 - y_hat))

    # We have to take cross entropy loss over all our samples, 100 in this 2-class iris dataset
    mean_cross_entropy_loss = torch.mean(cross_entropy_loss).detach().item()

    # Print our mean cross entropy loss
    if epoch % 20 == 0:
        print('Epoch {} | Loss: {}'.format(epoch, mean_cross_entropy_loss))
    loss_lst.append(mean_cross_entropy_loss)

    # (1) Forward propagation: to get our predictions to pass to our cross entropy loss function
    # (2) Back propagation: get our partial derivatives w.r.t. parameters (gradients)
    # (3) Gradient Descent: update our weights with our gradients
    model.train(X, y)

Epoch 0 | Loss: 0.7343553304672241
Epoch 20 | Loss: 0.6719828844070435
Epoch 40 | Loss: 0.6701794266700745
Epoch 60 | Loss: 0.6685028076171875
Epoch 80 | Loss: 0.6668243408203125
Epoch 100 | Loss: 0.6651425361633301
